In [1]:
import pandas as pd
import numpy as np
import glob
import os
import random
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight


In [2]:
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras as keras
from keras import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

Using plaidml.keras.backend backend.


In [3]:
enc = pickle.load( open( "pickle\\enc.p", "rb" ) )

In [4]:
#test_x = pickle.load( open( "pickle\\test_x.p", "rb" ) )
val_x = pickle.load( open( "pickle\\val_x.p", "rb" ) )
train_x = pickle.load( open( "pickle\\train_x.p", "rb" ) )
#test_labels = pickle.load( open( "pickle\\test_labels.p", "rb" ) )
val_labels = pickle.load( open( "pickle\\val_labels.p", "rb" ) )
train_labels = pickle.load( open( "pickle\\train_labels.p", "rb" ) )

In [5]:
print(train_x.shape)
print(len(train_labels))

(24445, 200, 27)
24445


In [6]:
class_weights = pickle.load( open( "pickle\\class_weights.p", "rb" ) )
class_weights

array([0.88338393, 1.02727349, 1.37085016, 1.48692214, 0.51480499,
       0.55160664, 3.36152365, 1.41072253, 0.72831009, 1.57547048,
       3.00454769, 0.71930909])

### Define Model

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM

In [8]:
train_x.shape[2]

27

In [9]:
train_x.shape

(24445, 200, 27)

In [10]:
out_index = 2
model = keras.Sequential()
model.add(LSTM(50, input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences = True))
model.add(LSTM(50, input_shape=(train_x.shape[1], train_x.shape[2]), dropout=.5))
#model.add(Dense(200, activation= "relu"))
#model.add(keras.layers.Dropout(.25))
model.add(Dense(len(enc.categories_[0]), activation= "softmax"))
model.compile(loss = "categorical_crossentropy", optimizer= "adam", metrics=["acc"])
print(model.summary())

INFO:plaidml:Opening device "opencl_amd_gfx804.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200, 50)           15600     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 12)                612       
Total params: 36,412
Trainable params: 36,412
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
type(train_x)
train_x.dtype

dtype('float64')

In [12]:
train_labels.shape

(24445, 12)

In [13]:
max(np.sum(train_labels, axis = 0))/sum(np.sum(train_labels, axis = 0))

0.1618735937819595

In [14]:
max(np.sum(val_labels, axis = 0))/sum(np.sum(val_labels, axis = 0))

0.1635617303441743

In [15]:
enc.categories_[0]

array(['AU', 'CO', 'EC', 'GB', 'HK', 'JP', 'MX', 'NZ', 'TN', 'TR', 'US',
       'ZA'], dtype=object)

In [ ]:
model.fit(train_x, train_labels,
          epochs = 50, 
          shuffle = True,
          validation_data = (val_x, val_labels),
          batch_size = 512,
          class_weight=class_weights)

Train on 24445 samples, validate on 6334 samples
Epoch 1/50


INFO:plaidml:Analyzing Ops: 1988 of 40247 operations complete
INFO:plaidml:Analyzing Ops: 4980 of 40247 operations complete
INFO:plaidml:Analyzing Ops: 8891 of 40247 operations complete
INFO:plaidml:Analyzing Ops: 11747 of 40247 operations complete
INFO:plaidml:Analyzing Ops: 15701 of 40247 operations complete
INFO:plaidml:Analyzing Ops: 21195 of 40247 operations complete
INFO:plaidml:Analyzing Ops: 25882 of 40247 operations complete
INFO:plaidml:Analyzing Ops: 32509 of 40247 operations complete
INFO:plaidml:Analyzing Ops: 35273 of 40247 operations complete
INFO:plaidml:Analyzing Ops: 39136 of 40247 operations complete


In [ ]:
model.get_layer(index = 0)